In [ ]:
import pandas as pd
import io

url='https://drive.google.com/file/d/1IjIEhLc9n8eLKeY-yh_YigKVWbhgGBsN/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

result_11 = df.loc[11::12, ['result','pick1', 'pick2', 'pick3', 'pick4', 'pick5']]
result_10 = df.loc[10::12, ['result','pick1', 'pick2', 'pick3', 'pick4', 'pick5']]

merged_result = pd.concat([result_10, result_11])
merged_result = merged_result.sort_index()
merged_result = merged_result.reset_index(level=None, drop=True)
merged_result

In [ ]:
# Drop rows with NaN values in any of the picks and their paired row
def clean_data(data):
    rows_to_drop = set()

    for i in range(0, len(data), 2):
        # Check both rows in the pair (team1 and team2)
        if data.iloc[i, 1:].isnull().any() or data.iloc[i + 1, 1:].isnull().any():
            rows_to_drop.add(i)
            rows_to_drop.add(i + 1)

    # Drop identified rows
    cleaned_data = data.drop(rows_to_drop).reset_index(drop=True)
    return cleaned_data

cleaned_data = clean_data(merged_result)


In [ ]:
data = cleaned_data

import pandas as pd
from gensim.models import Word2Vec

games = []
for i in range(0, len(data), 2):
    team1 = data.iloc[i, 1:6].tolist()
    team2 = data.iloc[i+1, 1:6].tolist()

    # Create sequences for both teams
    games.append(team1)  # Team 1
    games.append(team2)  # Team 2

# Train the Word2Vec model
model = Word2Vec(
    sentences=games,
    vector_size=8,
    window=5,
    min_count=1,
    sg=1
)

In [ ]:
model.save("champion_embeddings.model")
